In [5]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import random
import math
import pickle
import numpy as np
import copy

In [8]:
#This class generates correlated memory vectors as decribed in (Benna, Fusi; 2021)
class CorrelatedPatterns():
    def __init__(self, 
                 L, #Length of each memory vector 
                 p, #Number of ancestors
                 k, #Number of children per ancestor
                 gamma): #Average overlap between child and ancestor. A value of one means each child is identical to its ancestor,
                        #while a value of zero means each child is completely different from its ancestor.
        self.L = L
        self.p = p
        self.k = k
        self.gamma = gamma
        
        #Create three arrays to store the ancestor vectors, the descendant (child) vectors, and the difference vectors
        self.ancestors = []
        self.descendants = []
        self.differences = []
        
        #For purposes of PyTorch dataset creation, we will create two new lists that do not themselves contain lists
        self.descendants_singlelist = []
        self.differences_singlelist = []
        
        for _ancestorIndex in range(p):
            
            #Each ancestor is initialized randomly
            ancestor = np.random.choice((-1,1), size=(L))
            self.ancestors.append(np.array(ancestor))
            
            self.descendants.append([])
            #Initialize k descendants
            for _descendantIndex in range(k):
                descendant = torch.tensor([])
                for __i in range(len(ancestor)):
                    
                    #With probability 1-gamma, the descendant memory is corrupted at this bit. 
                    if(random.uniform(0,1) < 1-gamma):
                        descendant = torch.cat((descendant, (torch.tensor([ancestor[__i]]) * -1)))
                    else: #Otherwise, the ancestor's memory at this bit is copied to the descendant.
                        descendant = torch.cat((descendant, torch.tensor([ancestor[__i]])))
                
                #Save the memory
                self.descendants[_ancestorIndex].append(descendant.clone().detach())
                self.descendants_singlelist.append(descendant.clone().detach().reshape(1,-1))
            
            #Calculate the differences between the ancestor vectors and the child vectors
            self.differences.append([])
            for _descendantIndex in range(k):
                self.differences[_ancestorIndex].append(torch.tensor(self.ancestors[_ancestorIndex]) - self.descendants[_ancestorIndex][_descendantIndex])
                self.differences_singlelist.append((torch.tensor(self.ancestors[_ancestorIndex]) - self.descendants[_ancestorIndex][_descendantIndex]).reshape(1,-1))
                
        self.descendants_singlelist = torch.cat(self.descendants_singlelist)
        self.differences_singlelist = torch.cat(self.differences_singlelist)

#This subclass inherits the PyTorch Dataset class in order to create datasets of correlated memory.
class SensoryData(Dataset):
    def __init__(self, 
                 L,      #Length of each sample
                 p,      #Number of parents
                 k,      #Number of children per parent 
                 gamma   #Overlap between parent and children (1=identical, 0=no overlap)
                ):
        super().__init__()
        c = CorrelatedPatterns(L, p, k, gamma)
        memories = c.descendants_singlelist
        
        #Grab the memories generated by CorrelatedPatterns()
        self.data = memories
        self.x = memories
        self.y = memories
        self.n_samples = memories.shape[0]
    
    #Implement necessary helper functions
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [9]:
high_corr_data_len10 = SensoryData(10, 60, 100, 0.9)
torch.save(high_corr_data_len10, "high_corr_dataset_len10.pt")

In [290]:
med_corr_data_len10 = SensoryData(10, 60, 100, 0.6)
torch.save(med_corr_data_len10, "med_corr_dataset_len10.pt")

In [291]:
low_corr_data_len10 = SensoryData(10, 60, 100, 0.3)
torch.save(low_corr_data_len10, "low_corr_dataset_len10.pt")

In [10]:
class Autoencoder(nn.Module):
    def __init__(self, 
                 n_inputs, #Number of input units
                 n_hiddens): #Number of hidden units
        super().__init__()
        
        self.eweight = nn.Parameter(torch.rand(n_hiddens, n_inputs), requires_grad=True)
        #self.initial_weights = self.eweight.clone()
        self.initial_state_dict = copy.deepcopy(self.state_dict())
        
        
        self.n_inputs = n_inputs
        self.n_hiddens = n_hiddens
        
    #Implement the forward pass
    def forward(self, X):
        X = torch.flatten(X, start_dim=1)
        
        self.encoded = F.linear(X, self.eweight)
        self.hidden_activations = torch.relu(self.encoded)
        
        self.decoded = F.linear(self.hidden_activations, self.eweight.T)
        self.decoder_activations = torch.tanh(self.decoded)
        return self.decoder_activations, self.hidden_activations
    


In [11]:
loader = DataLoader(high_corr_data_len10, batch_size=1, shuffle=True)

focus_array = []
ARRAY_LEN = 10
for i in range(ARRAY_LEN):
    focus_array.append(next(iter(loader)))
    
def update_focus_array():
    pop_item = focus_array.pop(0)
    push_item = next(iter(loader))
    
    focus_array.append(push_item)
    
    return push_item

In [12]:
#Function written by Huidi Li
def compute_gradmask(model, grads, ratio=0.1): 
    masks = []
    for i, p in enumerate(model.parameters()):
        grads_shape = grads[i].shape
        grads_sorted, grads_sort_idx = torch.sort(torch.abs(grads[i]).flatten())
        min_idx = int(ratio * len(grads_sorted))
        mask = abs(grads[i])<grads_sorted[min_idx]
        masks.append(mask.reshape(grads_shape))
    return masks



In [13]:
def AddNoise(array):
    
    #mask_status = torch.rand(array.shape)
    #mask_status = mask_status >= torch.sort(mask_status.flatten()).values[-1*int(arr.nelement()*(bit_mask_pct))]
    #indices = np.random.permutation(np.arange(0,array.nelement(),1))
    #array[indices[0]] = 0
    #array[indices[1]] = 0
    random_index = random.randint(0,len(array))
    array[random_index % len(array)] = 0
    array[(random_index + 1) % len(array)] = 0
    
    return array

def train(
            model,
            optimizer,
            loss_function,
            outer_loop_epochs=1000,
            inner_loop_epochs=100, 
            alpha=0.5,
            reset_ratio=0.3
        ):
    
    initial_lr = optimizer.param_groups[0]['lr']
    model.train()
    
    initial_parameters = []
    for param_ind, param in enumerate(model.parameters()):
        initial_parameters.append(param)
    
    
    weight_history = {}
    
    for outer_epoch in range(outer_loop_epochs):
        
        e_item = update_focus_array()
        
        Xs = torch.Tensor()
        Ys = torch.Tensor()
        for i in range(len(focus_array)):
            Xs = torch.cat([Xs, focus_array[i][0]])
            Ys = torch.cat([Ys, focus_array[i][1]])
        
        model_gradients_ma = []
        
        
        W_old = []
        for param_ind, param in enumerate(model.parameters()):
            W_old.append(copy.deepcopy(param))
        
        
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [inner_loop_epochs*0.25,
                                                                      inner_loop_epochs*0.5],
                                                                gamma=0.1,
                                                                last_epoch=-1)
        optimizer.param_groups[0]['lr'] = initial_lr
        for inner_epoch in range(inner_loop_epochs):
            for _ in range(Xs.shape[0]):
                Xs[_] = AddNoise(Xs[_])
            
            predicted_y, net_hidden_activity = model(Xs)
            loss = loss_function(predicted_y, Ys)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            for param_ind, param in enumerate(model.parameters()):
                if(inner_epoch == 0):
                    model_gradients_ma.append(torch.abs(param.grad))
                else:
                    model_gradients_ma[param_ind] = alpha*torch.abs(param.grad) + (1-alpha)*model_gradients_ma[param_ind]
       
        
        W_new = []
        for param_ind, param in enumerate(model.parameters()):
            W_new.append(copy.deepcopy(param))
        
        #Save weights
        weight_history[outer_epoch] = {'e_item': e_item, 
                               'W_old': W_old,
                               'W_new': W_new, 
                               }
        
        #Reset weights
        mask = compute_gradmask(model, model_gradients_ma, ratio=reset_ratio)
        state_dict = model.state_dict()
        param_index = 0
        for param_name, param_value in state_dict.items():
            state_dict[param_name][mask[param_index]] = model.initial_state_dict[param_name][mask[param_index]]
            param_index += 1
            
        model.load_state_dict(state_dict)
        
    return weight_history      

In [14]:
loader = DataLoader(high_corr_data_len10, batch_size=1, shuffle=True)

focus_array = []
ARRAY_LEN = 10
for i in range(ARRAY_LEN):
    focus_array.append(next(iter(loader)))


model = Autoencoder(10,20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

loss_function = nn.MSELoss()

In [15]:
output_data = train(model=model,
      optimizer=optimizer,
      loss_function=loss_function,
      outer_loop_epochs=6000,
      inner_loop_epochs=64
     )

In [16]:
with open('1.pkl', 'wb') as f:
    pickle.dump(output_data, f)

In [391]:
loader = DataLoader(high_corr_data_len10, batch_size=1, shuffle=True)

focus_array = []
ARRAY_LEN = 30
for i in range(ARRAY_LEN):
    focus_array.append(next(iter(loader)))


model = Autoencoder(10,20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

loss_function = nn.MSELoss()

In [ ]:
output_data = train(model=model,
      optimizer=optimizer,
      loss_function=loss_function,
      outer_loop_epochs=6000,
      inner_loop_epochs=64
     )

In [ ]:
with open('4.pkl', 'wb') as f:
    pickle.dump(output_data, f)

In [ ]:
loader = DataLoader(high_corr_data_len10, batch_size=1, shuffle=True)

focus_array = []
ARRAY_LEN = 50
for i in range(ARRAY_LEN):
    focus_array.append(next(iter(loader)))


model = Autoencoder(10,20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

loss_function = nn.MSELoss()

In [ ]:
output_data = train(model=model,
      optimizer=optimizer,
      loss_function=loss_function,
      outer_loop_epochs=6000,
      inner_loop_epochs=64
     )

In [ ]:
with open('7.pkl', 'wb') as f:
    pickle.dump(output_data, f)

In [430]:
with open('4.pkl', 'rb') as f:
    o_data = pickle.load(f)

In [445]:
loader = DataLoader(med_corr_data_len10, batch_size=1, shuffle=True)

focus_array = []
ARRAY_LEN = 10
for i in range(ARRAY_LEN):
    focus_array.append(next(iter(loader)))


model = Autoencoder(10,20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

loss_function = nn.MSELoss()

In [446]:
output_data = train(model=model,
      optimizer=optimizer,
      loss_function=loss_function,
      outer_loop_epochs=6000,
      inner_loop_epochs=64
     )

In [447]:
with open('2.pkl', 'wb') as f:
    pickle.dump(output_data, f)

In [ ]:
loader = DataLoader(med_corr_data_len10, batch_size=1, shuffle=True)

focus_array = []
ARRAY_LEN = 30
for i in range(ARRAY_LEN):
    focus_array.append(next(iter(loader)))


model = Autoencoder(10,20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

loss_function = nn.MSELoss()

In [ ]:
output_data = train(model=model,
      optimizer=optimizer,
      loss_function=loss_function,
      outer_loop_epochs=6000,
      inner_loop_epochs=64
     )

In [ ]:
with open('5.pkl', 'wb') as f:
    pickle.dump(output_data, f)

In [ ]:
loader = DataLoader(med_corr_data_len10, batch_size=1, shuffle=True)

focus_array = []
ARRAY_LEN = 50
for i in range(ARRAY_LEN):
    focus_array.append(next(iter(loader)))


model = Autoencoder(10,20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

loss_function = nn.MSELoss()

In [ ]:
output_data = train(model=model,
      optimizer=optimizer,
      loss_function=loss_function,
      outer_loop_epochs=6000,
      inner_loop_epochs=64
     )

In [ ]:
with open('8.pkl', 'wb') as f:
    pickle.dump(output_data, f)

In [448]:
loader = DataLoader(low_corr_data_len10, batch_size=1, shuffle=True)

focus_array = []
ARRAY_LEN = 10
for i in range(ARRAY_LEN):
    focus_array.append(next(iter(loader)))


model = Autoencoder(10,20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

loss_function = nn.MSELoss()

In [449]:
output_data = train(model=model,
      optimizer=optimizer,
      loss_function=loss_function,
      outer_loop_epochs=6000,
      inner_loop_epochs=64
     )

In [450]:
with open('3.pkl', 'wb') as f:
    pickle.dump(output_data, f)

In [495]:
with open('1 (2) (1).pkl', 'rb') as f:
    output_data = pickle.load(f)

In [496]:
testmodel.eweight = output_data[5999]['W_new'][0]

In [497]:
def step_activation(array):
    arr = array
    arr[arr >= 0] = 1
    arr[arr < 0] = -1
    
    return arr

In [498]:
avg = 0
for i in range(5999):
    diff = step_activation(testmodel(output_data[i]['e_item'][0])[0]) - output_data[i]['e_item'][0]
    avg += torch.count_nonzero(diff)/10


In [499]:
avg/6000

tensor(0.3003)

In [ ]:
loader = DataLoader(low_corr_data_len10, batch_size=1, shuffle=True)

focus_array = []
ARRAY_LEN = 30
for i in range(ARRAY_LEN):
    focus_array.append(next(iter(loader)))


model = Autoencoder(10,20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

loss_function = nn.MSELoss()

In [ ]:
output_data = train(model=model,
      optimizer=optimizer,
      loss_function=loss_function,
      outer_loop_epochs=6000,
      inner_loop_epochs=64
     )

In [ ]:
with open('6.pkl', 'wb') as f:
    pickle.dump(output_data, f)

In [ ]:
loader = DataLoader(low_corr_data_len10, batch_size=1, shuffle=True)

focus_array = []
ARRAY_LEN = 50
for i in range(ARRAY_LEN):
    focus_array.append(next(iter(loader)))


model = Autoencoder(10,20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

loss_function = nn.MSELoss()

In [ ]:
output_data = train(model=model,
      optimizer=optimizer,
      loss_function=loss_function,
      outer_loop_epochs=600,
      inner_loop_epochs=64
     )

In [ ]:
with open('9.pkl', 'wb') as f:
    pickle.dump(output_data, f)